# Análisis de serie de tiempo con ARIMA

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />

ARIMA o Modelo autorregresivo integrado de media móvil es un modelo estadístico que utiliza variaciones y regresiones de datos estadísticos con el fin de encontrar patrones para una predicción hacia el futuro. Se trata de un modelo dinámico de series temporales, es decir, las estimaciones futuras vienen explicadas por los datos del pasado y no por variables independientes.

Usaremos la librería [Statsmodels](https://www.statsmodels.org) para hacer análisis estadístico.

## Importar librerías

In [ ]:
# Import libraries
import warnings
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Defaults
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')

Obtenemos los datos clásicos de pasajeros de aerolíneas internacionales, descargables desde la página web de DataMarket (https://datamarket.com/data/set/22u3/international-airline-passengers-monthly-totals-in-thousands-jan-49-dec-60#!ds=22u3&display=line)


In [ ]:
# Load the data
data_df = pd.read_csv('data/international-airline-passengers.csv', engine='python', skipfooter=3)
data_df.head()

In [ ]:
# Convertimos la columna month en datetime
data_df['Month']=pd.to_datetime(data_df['Month'], format='%Y-%m-%d')
data_df.set_index(['Month'], inplace=True)

# Graficamos los datos
data_df.plot()
plt.ylabel('Monthly airline passengers (x1000)')
plt.xlabel('Date')
plt.show()

Aparecen dos patrones obvios en los datos: un aumento general en el número de pasajeros a lo largo del tiempo y una estacionalidad de 12 meses con picos correspondientes al período de verano del hemisferio norte.

## ARIMA
ARIMA significa Modelo autorregresivo integrado de media móvil. Hay tres enteros (p, d, q) que se utilizan para parametrizar los modelos ARIMA. Por eso, un modelo ARIMA no estacional se denota con ARIMA (p, d, q):
<ul>
<li> <strong> p </strong> es el número de términos autorregresivos (parte AR). Permite incorporar el efecto de valores pasados ​​en nuestro modelo. Intuitivamente, esto sería similar a afirmar que es probable que haga calor mañana si ha estado caliente los últimos 3 días. </li>
<li> <strong> d </strong> es el número de diferencias no estacionales necesarias para la estacionariedad. Intuitivamente, esto sería similar a afirmar que es probable que sea la misma temperatura mañana si la diferencia de temperatura en los últimos tres días ha sido muy pequeña. </li>
<li> <strong> q </strong> es el número de errores de pronóstico rezagados en la ecuación de predicción (parte MA). Esto nos permite establecer el error de nuestro modelo como una combinación lineal de los valores de error observados en puntos de tiempo anteriores en el pasado. </li>
</ul>

Cuando se trata de efectos estacionales, como en nuestro ejemplo, se usa ARIMA estacional, que se denota como ARIMA (p, d, q) (P, D, Q) s. Aquí, (p, d, q) son los parámetros no estacionales descritos anteriormente, (<strong> P, D, Q </strong>) siguen la misma definición pero se aplican al componente estacional de la serie de tiempo. El término <strong> s </strong> es la periodicidad de la serie temporal.

Si bien en este caso está claro que s = 12, ¿cómo establecemos los otros parámetros?

Se basa más o menos en la experiencia. Existen numerosas prácticas recomendadas que se pueden seguir para identificar los modelos ARIMA, como:
http://people.duke.edu/~rnau/arimrule.htm.

Aquí utilizamos la búsqueda de cuadrícula sobre todas las combinaciones posibles de valores de parámetros dentro de un rango predefinido de valores (fuertemente inspirado en https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3).

$ statsmodels.tsa.statespace.sarimax.SARIMAXResults$ devuelve valores para AIC (Criterio de información de Akaike) y BIC (Criterio de información de Bayes) que se pueden minimizar para seleccionar el mejor modelo de ajuste. Usamos el valor AIC, que estima la información perdida cuando se usa un modelo dado para representar el proceso que genera los datos. Al hacerlo, se trata de la compensación entre la precisión de ajuste del modelo y la complejidad del modelo en sí.

In [ ]:
# Define the d and q parameters to take any value between 0 and 1
q = d = range(0, 2)
# Define the p parameters to take any value between 0 and 3
p = range(0, 4)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

print(pdq)

In [ ]:
# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Seleccionamos un subconjunto de la serie de datos como datos de entrenamiento, digamos los primeros 11 años. Nuestro objetivo es predecir el último año de la serie en función de esta información.

In [ ]:
train_data = data['1949-01-01':'1959-12-01']
test_data = data['1960-01-01':'1960-12-01']

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

AIC = []
SARIMAX_model = []
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMAX{}x{} - AIC:{}'.format(param, param_seasonal, results.aic), end='\r')
            AIC.append(results.aic)
            SARIMAX_model.append([param, param_seasonal])
        except:
            continue

In [ ]:
print('El valor de AIC más pequeño es {} para el modelo SARIMAX{}x{}'.format(min(AIC), SARIMAX_model[AIC.index(min(AIC))][0],SARIMAX_model[AIC.index(min(AIC))][1]))


In [ ]:
# Ajustemos el modelo
mod = sm.tsa.statespace.SARIMAX(train_data,
                                order=SARIMAX_model[AIC.index(min(AIC))][0],
                                seasonal_order=SARIMAX_model[AIC.index(min(AIC))][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

In [ ]:
results.summary()

Una vez que el modelo ha sido ajustado, podemos verificar si hace lo que esperamos y si se violan los supuestos que hicimos. Para hacer esto, podemos usar el método $ plot\_diagnostics $.

In [ ]:
results.plot_diagnostics(figsize=(20, 14))
plt.show()

En las gráficas anteriores, podemos observar que los residuos no están correlacionados (gráfica inferior derecha) y no exhiben ninguna estacionalidad obvia (la gráfica superior izquierda). Además, los residuos y aproximadamente distribuidos normalmente con media cero (gráfico superior derecho). El gráfico qq en la parte inferior izquierda muestra que la distribución ordenada de los residuos (puntos azules) sigue fielmente la tendencia lineal de las muestras tomadas de una distribución normal estándar con N (0, 1). Nuevamente, esta es una fuerte indicación de que los residuos se distribuyen normalmente.

## Resultados

Haremos algunas predicciones. Usaremos tres métodos,

1. En una predicción de muestra con predicción de 1 paso del año pasado (1959). En este caso, el modelo se usa para predecir datos sobre los que se construyó el modelo. El pronóstico de 1 paso adelante implica que cada punto pronosticado se usa para predecir el siguiente.

In [ ]:
pred0 = results.get_prediction(start='1958-01-01', dynamic=False)
pred0_ci = pred0.conf_int()

2. En predicción de muestra con pronóstico dinámico del último año (1959). Nuevamente, el modelo se usa para predecir datos sobre los que se construyó el modelo.

In [ ]:
pred1 = results.get_prediction(start='1958-01-01', dynamic=True)
pred1_ci = pred1.conf_int()

3. Previsión "verdadera" de datos fuera de la muestra. En este caso, se le pide al modelo que prediga datos que no ha visto antes.

In [ ]:
pred2 = results.get_forecast('1962-12-01')
pred2_ci = pred2.conf_int()

print(pred2.predicted_mean['1960-01-01':'1960-12-01'])

Grafiquemos estas tres predicciones

In [ ]:
ax = data.plot(figsize=(20, 16))
pred0.predicted_mean.plot(ax=ax, label='1-step-ahead Forecast (get_predictions, dynamic=False)')
pred1.predicted_mean.plot(ax=ax, label='Dynamic Forecast (get_predictions, dynamic=True)')
pred2.predicted_mean.plot(ax=ax, label='Dynamic Forecast (get_forecast)')
ax.fill_between(pred2_ci.index, pred2_ci.iloc[:, 0], pred2_ci.iloc[:, 1], color='k', alpha=.1)
plt.ylabel('Monthly airline passengers (x1000)')
plt.xlabel('Date')
plt.legend()
plt.show()

Mirando la figura, el modelo parece hacer un muy buen trabajo al modelar la serie temporal. Las líneas azules y púrpuras son, como se esperaba, muy cercanas a la verdad del fondo rojo. Lo que es más interesante es la línea gris, la predincción fuera de la muestra. Para una serie temporal tan simple, el modelo ARIMA puede pronosticar los valores de 1960 con precisión.

Para cuantificar la precisión de la predicción para 1960, podemos calcular métricas como Error absoluto medio (MAE), Error cuadrado medio (MSE) o Error cuadrado medio raíz (RMSE).

Estas son todas métricas absolutas, por lo tanto, dependen de la escala. Si bien son útiles y se usan ampliamente para comparar diferentes métodos en un mismo conjunto de datos, aquí sería más útil expresar el error en relación con la magnitud de la serie de tiempo que estamos tratando de predecir. Una medida de error porcentual de uso común es el error porcentual absoluto medio (MAPE). Tiene algunos inconvenientes en casos especiales (en particular, puede conducir a la división por cero) y se han propuesto medidas mejoradas, ver p. Ej. https://www.otexts.org/fpp/2/5 para una buena visión general. Sin embargo, para este ejemplo nos quedaremos con MAPE.

In [ ]:
prediction = pred2.predicted_mean['1960-01-01':'1960-12-01'].values
# flatten nested list
truth = list(itertools.chain.from_iterable(test_data.values))
# Mean Absolute Percentage Error
MAPE = np.mean(np.abs((truth - prediction) / truth)) * 100

print('The Mean Absolute Percentage Error for the forecast of year 1960 is {:.2f}%'.format(MAPE))